<a href="https://colab.research.google.com/github/kashishkhera44/comp215/blob/main/labs/lab02-review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

COMP 215 - LAB 2
----------------
#### Name(s):Kashish Khera
#### Date: 15-01-2025

By the end of this lab you should be able to:
  * use strings, tuples, lists, and dictionaries (review)
  * use *f-string* to simplify string formatting operations
  * write functions and simple unit tests (review)
  * use *list comprehension* to implement map and filter algorithms


During this lab, you will be introduced to the following:
  * API Keys
  * JSON data formats
  * the `datetime` module to represent dates and times
  * the `pprint` module to format data structures

## Near Earth Objects

In this lab, we'll answer some questions about [Near Earth Objects](https://cneos.jpl.nasa.gov/) using NASA's API:  [https://api.nasa.gov/](https://api.nasa.gov/#NeoWS).  But, as usual, let's do the imports first.


In [1]:
import datetime, json, requests
from pprint import pprint    # Pretty Print - built-in python function to nicely format data structures

### dates & f-strings

First we need a short tutorial on python dates and [f-strings](https://realpython.com/python-f-strings/)...

In [2]:
today = datetime.date.today()   # get a date object representing today's date
print(today, type(today))
formatted_date = f'Today is: {today}'   # A format string - notice how the variable `today` is formatted into the string
print(formatted_date)

2025-01-15 <class 'datetime.date'>
Today is: 2025-01-15


### Make an API Query

Let's get some data from the NEO database; here's a query that gets the observation "feed" for today.  Notice the first line of the block is an 'API key'.  An API key is a unique identifier that authenticates a user when making a request to an API.  Run the block below using the API key 'DEMO_KEY' (just to be sure it works), then go to https://api.nasa.gov and get your own API key.  Use your own API key for the remainder of the lab.

In [33]:
API_KEY = 'aMyvm07dkmDrH0Lktiwf5JUg7srNobWQ5BkdVYOY'  # substitute your API key here

# get today's date (as a string)
#today = str(datetime.date.today())
today = '2025-01-15'

# use an f-string to "format" the date and API key varaibles.
url = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={today}&end_date={today}&api_key={API_KEY}'

# make the API request (recall from lab 1)
response = requests.request("GET", url, headers={}, data={})

# convert the response to json format (recall from lab 1)
data = json.loads(response.text)

# TIP: print(data) to see the whole data structure returned, here we grab just the list of NEO's:
n_results = data['element_count']
neos = data['near_earth_objects'][today]
print(f'{n_results} Near Earth Objects found for {today}')
pprint(neos[:1])

12 Near Earth Objects found for 2025-01-15
[{'absolute_magnitude_h': 20.83,
  'close_approach_data': [{'close_approach_date': '2025-01-15',
                           'close_approach_date_full': '2025-Jan-15 21:12',
                           'epoch_date_close_approach': 1736975520000,
                           'miss_distance': {'astronomical': '0.1411992421',
                                             'kilometers': '21123105.863774327',
                                             'lunar': '54.9265051769',
                                             'miles': '13125289.3559946326'},
                           'orbiting_body': 'Earth',
                           'relative_velocity': {'kilometers_per_hour': '39846.7042336836',
                                                 'kilometers_per_second': '11.0685289538',
                                                 'miles_per_hour': '24759.1924167575'}}],
  'estimated_diameter': {'feet': {'estimated_diameter_max': 1330.5310992191,
   

Next we extract just the potentially hazerdous asteroids, using a Comp115-style list accumulator *loop*:

In [28]:
hazards =  []
for item in neos:
  if item['is_potentially_hazardous_asteroid'] is True:
    hazards.append(item)
print(f'{len(hazards)} potentially hazardous asteroids identified.')

3 potentially hazardous asteroids identified.


### Fetch Complete Data for One Asteroid

Notice that the record for each `neo` is a dictionary with `id` field that uniquely identifies this record in the database.  We can use this `id` to fetch complete orbital and close approach data for the NEO.  For example, this query fetches the complete data set for the first hazardous asteroid...  Notice that the `miss_distance` field contains the distance (in various units) by which the NEO missed an "orbiting body".


In [63]:
id = hazards[0]['id']
url = f'https://api.nasa.gov/neo/rest/v1/neo/{id}?api_key={API_KEY}'
response = requests.request("GET", url, headers={}, data={})
data = json.loads(response.text)

pprint(data)

IndexError: list index out of range

## Exercise 1

In the code cell below, **re-write the accumulator loop** that creates the list of hazards as a [list comprehension](https://realpython.com/lessons/list-comprehensions-overview/) that implements a ["filter"](https://youtu.be/hUes6y2b--0)
Notice how this provides a concise way to "filter" items of interest from a larger data set.

In [35]:
# Ex. 1 your code here

hazards =[item for item in neos if item['is_potentially_hazardous_asteroid'] ]
print(f'{len(hazards)} potentially hazardous asteroids identified. ')

0 potentially hazardous asteroids identified. 




## Exercise 2

In the code cell below, write a python function that takes a list of "close approach data" as a parameter,
and returns a 2-tuple with the (date, miss km) of the closest approach to Earth in the list (where "miss km" is the miss distance in km).

Hints:
* notice the input is a list of dictionaries.  Each dictionary has a 'close_approach_date", "orbiting_body", and 'miss_distance' field.
* we are only interested in the closest approach to "Earth"


Add at least one unit test to check your work - note the test data only needs dictionaries with the fields your function actually uses.


In [64]:
def closest_approach(close_approach_data):
    """
    Finds the closest approach to Earth in the given data.

    Parameters:
        close_approach_data (list): A list of dictionaries containing 'close_approach_date',
                                    'orbiting_body', and 'miss_distance'.

    Returns:
        tuple: A tuple with the (date, miss distance in km) of the closest approach to Earth.
    """
    # Extract Earth approaches with relevant data
    earth_approaches = [
        (entry['close_approach_date'], float(entry['miss_distance']))
        for entry in close_approach_data if entry['orbiting_body'].lower() == 'earth'
    ]

    # Print the filtered earth_approaches for debugging
    print("Earth Approaches:", earth_approaches)

    # Return the closest approach using min()
    return min(earth_approaches, key=lambda x: x[1])


# Example Data for Testing
neos = data['near_earth_objects'][today]

# Run the function with the test data
closest_date, closest_distance = closest_approach(neos)



KeyError: 'orbiting_body'

## Challenge Exercise

In the code cell below, write a complete program that:
 1. fetches the list of NEO's for this week.
 2. for each NEO, fetch it's complete orbital data and determine its closest approach to Earth
 3. identify which NEO from this week's data makes the closet approach to earth
 4. print a nice message with information about the NEO, which it will approach the Earth, and how close it will come.

Hints:
* you'll need the start and end date - end date is today, see if you can use a [`timedelta`](https://docs.python.org/3/library/datetime.html#timedelta-objects) object to ge the start date (you can do basic "date math" with `timedelta` and `date` objects!)
* you may need to modify the function we wrote in Ex. 2 to return a triple with the NEO's id included;
* lots of opportunity here for more practice with list comprehensions


In [66]:
# Ex. 3 (challenge) your code here
import requests
from datetime import datetime, timedelta

# Get today's date and calculate start date (7 days ago)
end_date = datetime.today().date()
start_date = end_date - timedelta(days=7)

# API endpoint to fetch NEOs for the given date range
url = f"https://api.nasa.gov/neo/rest/v1/neo/browse?start_date={start_date}&end_date={end_date}&api_key=aMyvm07dkmDrH0Lktiwf5JUg7srNobWQ5BkdVYOY"

# Fetch the data
response = requests.get(url)
data = response.json()

# Function to fetch orbital data for a specific NEO
def get_orbital_data(neo_id):
    url = f"https://api.nasa.gov/neo/rest/v1/neo/{neo_id}?api_key=DEMO_KEY"
    response = requests.get(url)
    return response.json()

# List to hold NEO information
neo_info = []

# For each NEO, fetch orbital data and calculate closest approach to Earth
for neo in data['near_earth_objects']:
    neo_id = neo['id']
    orbital_data = get_orbital_data(neo_id)

    # Get the closest approach date and distance from Earth
    closest_approach = orbital_data['close_approach_data'][0]
    approach_date = closest_approach['close_approach_date']
    distance = closest_approach['miss_distance']['kilometers']

    # Append the data to the list
    neo_info.append((neo['name'], approach_date, distance, neo_id))

# Identify the NEO that makes the closest approach to Earth
closest_neo = min(neo_info, key=lambda x: float(x[2]))

# Print the result
neo_name, approach_date, distance, neo_id = closest_neo
print(f"The NEO {neo_name} will make its closest approach to Earth on {approach_date}.")
print(f"It will come as close as {distance} kilometers.")

KeyError: 'close_approach_data'